# Tic-Tac-Toe (Events + Observers)

This notebook uses event-style inputs with a `events` namespace.
Human plays X, AI plays O. The widget emits outputs, and Python responds by issuing events.

In [2]:
import time
import vibe_widget as vw

## Create the game widget

In [3]:
game_board = vw.create(
    """Interactive Tic-Tac-Toe game board
    - Human plays X, AI plays O
    - Click cells to make moves
    - Outputs board_state, current_turn, game_over
    - Listen for custom events: ai_move(index: 0-8, row-major)
    - Handle events via model.on('msg:custom', ...) and apply ai_move
    """,
    outputs=vw.outputs(
        board_state="9-element array of 'x', 'o', or 'b'",
        game_over="boolean",
        current_turn="'x' or 'o'"
    ),
    events=vw.events(
        ai_move=vw.event(
            "index 0-8 (row-major) for the AI's move",
            params={"index": "0-8 row-major"}
        )
    ),
    cache=False
)

game_board

DynamicVibeWidget(board_state=<VibeExport Interactive Tic-Tac-Toe game board
    - Human plays X, AI plays O
 ...

DynamicVibeWidget(board_state=<VibeExport interactive_tic_tac_toe_game_board_human.board_state>, code='import ...

In [6]:
game_board.code

'import * as d3 from "https://esm.sh/d3@7";\n\n/**\n * Tic-Tac-Toe Cell Component\n */\nexport const Cell = ({ value, onClick, index, disabled, html }) => {\n  const displayValue = value === \'x\' ? \'X\' : value === \'o\' ? \'O\' : \'\';\n  const color = value === \'x\' ? \'#2563eb\' : \'#dc2626\';\n  \n  return html`\n    <button \n      onClick=${() => !disabled && value === \'b\' && onClick(index)}\n      style=${{\n        width: \'80px\',\n        height: \'80px\',\n        fontSize: \'2rem\',\n        fontWeight: \'bold\',\n        display: \'flex\',\n        alignItems: \'center\',\n        justifyContent: \'center\',\n        backgroundColor: \'#ffffff\',\n        border: \'2px solid #e2e8f0\',\n        borderRadius: \'8px\',\n        cursor: disabled || value !== \'b\' ? \'not-allowed\' : \'pointer\',\n        color: color,\n        transition: \'all 0.2s ease\'\n      }}\n    >\n      ${displayValue}\n    </button>\n  `;\n};\n\n/**\n * Main Widget\n */\nexport default functi

## AI move logic

In [4]:
def pick_first_empty(board_list):
    for idx, cell in enumerate(board_list):
        if cell == "b":
            return idx
    return None

## Observe outputs and issue events

In [5]:
def on_turn_change(event):
    if event.new != "o":
        return

    # Allow the frontend to finish updating its state.
    time.sleep(0.1)

    board_state = game_board.outputs.board_state.value
    if not board_state or game_board.outputs.game_over.value:
        return

    if isinstance(board_state, str):
        import ast
        board_state = ast.literal_eval(board_state)

    board_list = list(board_state)
    if len(board_list) != 9:
        return

    move_index = pick_first_empty(board_list)
    if move_index is None:
        return

    game_board.events.ai_move(index=move_index)


game_board.outputs.current_turn.observe(on_turn_change)
